In [1]:
!pip install torchmetrics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 841.5/841.5 kB 8.1 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-many

In [2]:
!pip install bert_score

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 2.4 MB/s eta 0:00:00


In [3]:
import os
import time
import csv
import json
import numpy as np
import torch as th
import glob

import nltk
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from bert_score import score

os.environ['CUDA_VISIBLE_DEVICES']='0'

In [4]:
from torchmetrics.text.rouge import ROUGEScore

In [5]:
def get_bleu(recover, reference):
    return sentence_bleu([recover.split()], reference.split(), smoothing_function=SmoothingFunction().method4,)

def selectBest(sentences):
    selfBleu = [[] for i in range(len(sentences))]
    for i, s1 in enumerate(sentences):
        for j, s2 in enumerate(sentences):
            score = get_bleu(s1, s2)
            selfBleu[i].append(score)
    for i, s1 in enumerate(sentences):
        selfBleu[i][i] = 0
    idx = np.argmax(np.sum(selfBleu, -1))
    return sentences[idx]

def diversityOfSet(sentences):
    selfBleu = []
    # print(sentences)
    for i, sentence in enumerate(sentences):
        for j in range(i+1, len(sentences)):
            # print(sentence, sentences[j])
            score = get_bleu(sentence, sentences[j])
            selfBleu.append(score)
    if len(selfBleu)==0:
        selfBleu.append(0)
    div4 = distinct_n_gram_inter_sent(sentences, 4)
    return np.mean(selfBleu), div4


def distinct_n_gram(hypn,n):
    dist_list = []
    for hyp in hypn:
        hyp_ngrams = []
        hyp_ngrams += nltk.ngrams(hyp.split(), n)
        total_ngrams = len(hyp_ngrams)
        unique_ngrams = len(list(set(hyp_ngrams)))
        if total_ngrams == 0:
            return 0
        dist_list.append(unique_ngrams/total_ngrams)
    return  np.mean(dist_list)


def distinct_n_gram_inter_sent(hypn, n):
    hyp_ngrams = []
    for hyp in hypn:
        hyp_ngrams += nltk.ngrams(hyp.split(), n)
    total_ngrams = len(hyp_ngrams)
    unique_ngrams = len(list(set(hyp_ngrams)))
    if total_ngrams == 0:
        return 0
    dist_n = unique_ngrams/total_ngrams
    return  dist_n

def metrics_calculate(recovers, sources, references):
    bleu = []
    rougel = []
    avg_len = []
    dist1 = []

    sos = '[CLS]'
    eos = '[SEP]'
    sep = '[SEP]'
    pad = '[PAD]'

    rougeScore = ROUGEScore()

    for i in range(len(sources)):
        sources[i] = sources[i].replace(eos, '').replace(sos, '')
        references[i] = references[i].replace(eos, '').replace(sos, '').replace(sep, '')
        recovers[i] = recovers[i].replace(eos, '').replace(sos, '').replace(sep, '').replace(pad, '')

        avg_len.append(len(recovers[i].split(' ')))
        bleu.append(get_bleu(recovers[i], references[i]))
        rougel.append(rougeScore(recovers[i], references[i])['rougeL_fmeasure'].tolist())
        dist1.append(distinct_n_gram([recovers[i]], 1))

    P, R, F1 = score(recovers, references, model_type='microsoft/deberta-xlarge-mnli', lang='en', verbose=True)

    bleu = np.mean(bleu)
    rougel = np.mean(rougel)
    F1 = th.mean(F1)
    dist1 = np.mean(dist1)
    avg_len = np.mean(avg_len)
    # print('avg BLEU score', bleu)
    # print('avg ROUGE-L score', rougel)
    # print('avg berscore', F1)
    # print('avg dist1 score', dist1)
    # print('avg len',avg_len)
    return bleu, rougel, F1, dist1, avg_len

def metrics_calculate_S(recovers_S, sources_S, references_Srecovers_S, S):
    bleu = []
    rougel = []
    avg_len = []
    dist1 = []

    sos = '[CLS]'
    eos = '[SEP]'
    sep = '[SEP]'
    pad = '[PAD]'

    rougeScore = ROUGEScore()

    sentenceDict = {}
    referenceDict = {}
    sourceDict = {}
    for i in range(len(sources_S[0])):
        sentenceDict[i] = []
        referenceDict[i] = []
        sourceDict[i] = []

    for s in range(S):
        sources = sources_S.pop()
        references = references_S.pop()
        recovers = recovers_S.pop()
        for i in range(len(sources)):
            sources[i] = sources[i].replace(eos, '').replace(sos, '')
            references[i] = references[i].replace(eos, '').replace(sos, '').replace(sep, '')
            recovers[i] = recovers[i].replace(eos, '').replace(sos, '').replace(sep, '').replace(pad, '')

            sentenceDict[i].append(recovers[i])
            referenceDict[i].append(references[i])
            sourceDict[i].append(sources[i])

    # diversity
    div4 = []
    selfBleu = []
    for k, v in sentenceDict.items():
        if len(v) == 0:
            continue
        sb, d4 = diversityOfSet(v)
        selfBleu.append(sb)
        div4.append(d4)

    selfBleu = np.mean(selfBleu)
    div4 = np.mean(div4)
    # print('avg selfBleu score', selfBleu)
    # print('avg div4 score', div4)

    bleu = []
    rougel = []
    avg_len = []
    dist1 = []
    recovers = []
    references = []
    sources = []

    for k, v in sentenceDict.items():
        if len(v) == 0 or len(referenceDict[k]) == 0:
            continue

        recovers.append(selectBest(v))
        references.append(referenceDict[k][0])
        sources.append(sourceDict[k][0])

    for (source, reference, recover) in zip(sources, references, recovers):
        bleu.append(get_bleu(recover, reference))
        rougel.append(rougeScore(recover, reference)['rougeL_fmeasure'].tolist())
        avg_len.append(len(recover.split(' ')))
        dist1.append(distinct_n_gram([recover], 1))

    P, R, F1 = score(recovers, references, model_type='microsoft/deberta-xlarge-mnli', lang='en', verbose=True)

    bleu = np.mean(bleu)
    rougel = np.mean(rougel)
    F1 = th.mean(F1)
    dist1 = np.mean(dist1)
    avg_len = np.mean(avg_len)
    # print('avg BLEU score', bleu)
    # print('avg ROUGE-L score', rougel)
    # print('avg berscore', F1)
    # print('avg dist1 score', dist1)
    return bleu, rougel, F1, dist1, selfBleu, div4, avg_len

def create_log_file(filename):
    with open(f'score/{filename}.csv', 'w' , newline='') as csvfile:
        writer = csv.writer(csvfile)

def record_metric_score(msg, filename):
    with open(f'score/{filename}.csv', 'a', newline='') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow([msg])

In [13]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [16]:
# ver = 'Rdiff'
ver = 'Rsparse'
# ver = 'Rscale'
# ver = 'beta_ins_onlyZero'
# ver = 'onlyD'
checkpoint_path = f'/content/drive/MyDrive/RD資料夾/DiffusionOutput/BalaGinjo_checkpoint/{ver}'

bleu_list = list()
rougel_list = list()
bert_score_list = list()
dist1_list = list()
selfBleu_list = list()
div4_list = list()
avg_len_list = list()

# for Gu check
# 1. epochs, 2. use_ema
epochs = [i*1000 for i in range(30,31+1)]
use_ema = True
S = 1

s_t = time.time()
for epoch in epochs:
  if use_ema:
      filepath = f'{checkpoint_path}/test_inferences_ema_{epoch}.csv'
  else:
      filepath = f'{checkpoint_path}/test_inferences_{epoch}.csv'
  print(filepath)

  information = {}
  with open(filepath, 'r', newline='') as csvfile:
      rows = csv.reader(csvfile)
      recovers = []
      sources = []
      references = []
      for row in rows:
          if f'Epoch' in row[0]:
              epoch = row[0].replace('\n', '')
              continue
          temp = row[0].replace('\n', '').replace('  ', '').replace('\'', '').replace('\"', '').split(',source:')
          recovers.append(temp[0].split('recover:')[1])
          sources.append(temp[1].split(',reference:')[0])
          references.append(temp[1].split(',reference:')[1])
      information[0] = [epoch, recovers, sources, references]
  print(len(recovers), len(sources), len(references))

  recovers_S = list()
  sources_S = list()
  references_S = list()
  for i in range(len(information)):
      epoch = information[i][0]
      recovers = information[i][1]
      sources = information[i][2]
      references = information[i][3]

      recovers_S.append(recovers)
      sources_S.append(sources)
      references_S.append(references)

  bleu, rougel, bert_score, dist1, selfBleu, div4, avg_len = metrics_calculate_S(recovers_S, sources_S, references_S, S)
  bleu_list.append(bleu)
  rougel_list.append(rougel)
  bert_score_list.append(bert_score)
  dist1_list.append(dist1)
  selfBleu_list.append(selfBleu)
  div4_list.append(div4)
  avg_len_list.append(avg_len)

  del recovers_S, sources_S, references_S, information, recovers, sources, references

e_t = time.time() - s_t
print(f'[Evaluate] Using {e_t/60:.2f} mins.')

/content/drive/MyDrive/RD資料夾/DiffusionOutput/BalaGinjo_checkpoint/Rsparse/test_inferences_ema_30000.csv
128 128 128
calculating scores...
computing bert embedding.


  0%|          | 0/4 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/2 [00:00<?, ?it/s]

done in 2.60 seconds, 49.27 sentences/sec
/content/drive/MyDrive/RD資料夾/DiffusionOutput/BalaGinjo_checkpoint/Rsparse/test_inferences_ema_31000.csv
128 128 128
calculating scores...
computing bert embedding.


  0%|          | 0/4 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/2 [00:00<?, ?it/s]

done in 2.47 seconds, 51.72 sentences/sec
[Evaluate] Using 0.85 mins.


In [17]:
for i in range(len(epochs)):
  print(f'epoch: {epochs[i]}')
  print(f'bleu: {bleu_list[i]:.4f},\nrougel: {rougel_list[i]:.4f},\nbert score: {bert_score_list[i]:.4f},\ndist1: {dist1_list[i]:.4f},\nselfBleu: {selfBleu_list[i]:.4f},\ndiv4: {div4_list[i]:.4f},\navg_len: {avg_len_list[i]:.2f}')
  print()

epoch: 30000
bleu: 0.0081,
rougel: 0.0895,
bert score: 0.4615,
dist1: 0.8415,
selfBleu: 0.0000,
div4: 1.0000,
avg_len: 22.52

epoch: 31000
bleu: 0.0083,
rougel: 0.0875,
bert score: 0.4699,
dist1: 0.8461,
selfBleu: 0.0000,
div4: 1.0000,
avg_len: 21.98



In [ ]:
# for Gu check MBR ver
# MBR
# ver = 'Rdiff'
ver = 'Rsparse'
# ver = 'Rscale'
# ver = 'beta_ins_onlyZero'
# ver = 'onlyD'
checkpoint_path = f'/content/drive/MyDrive/RD資料夾/DiffusionOutput/BalaGinjo_checkpoint/{ver}'

bleu_list = list()
rougel_list = list()
bert_score_list = list()
dist1_list = list()
selfBleu_list = list()
div4_list = list()
avg_len_list = list()

# for Gu check
# 1. epochs, 2. use_ema
epochs = [i*1000 for i in range(29, 29+1)]
use_ema = False
# seed_list = [104, 66, 999, 168, 87, 555, 97, 888, 777, 108]
# seed_list = [6, 9, 525, 64, 25, 52, 95, 65, 50, 7]
seed_list = [640, 601, 949, 383, 821, 367, 248, 872, 963, 577]
# seed_list = [3, 46, 51, 77, 30, 761, 707, 210, 108, 208]
# seed_list = [221, 417, 1222, 619, 723, 309, 516, 1020, 614, 113]
# seed_list = [1996, 717, 17, 71, 1997, 218, 2, 18, 21, 8]
# seed_list = [86, 10, 8, 84, 3, 26, 82, 11, 85, 2]
# seed_list = [50, 65, 821, 6, 640, 9, 525, 601, 25, 52, 7, 963, 64, 383, 248]
# seed_list = [1308, 7076, 9567, 7292, 8136, 6323, 6139, 8870, 6853, 2817]
# seed_list = [49, 50, 53, 65, 821]
S = len(seed_list)

s_t = time.time()

hard_prompt = ''
filepath = list()
for seed in seed_list:
  if use_ema:
    filepath.append(f'{checkpoint_path}/test_inferences_ema_{epochs[0]}_seed{seed}{hard_prompt}.csv')
  else:
    filepath.append(f'{checkpoint_path}/test_inferences_{epochs[0]}_seed{seed}{hard_prompt}.csv')
print(len(filepath), filepath)

information = {}
count = 0
for file in filepath:
  with open(file, 'r', newline='') as csvfile:
      rows = csv.reader(csvfile)
      recovers = []
      sources = []
      references = []
      for row in rows:
          if f'Epoch' in row[0]:
              epoch = row[0].replace('\n', '')
              continue
          temp = row[0].replace('\n', '').replace('  ', '').replace('\'', '').replace('\"', '').split(',source:')
          recovers.append(temp[0].split('recover:')[1])
          sources.append(temp[1].split(',reference:')[0])
          references.append(temp[1].split(',reference:')[1])
      information[count] = [epoch, recovers, sources, references]
      count += 1
      print(len(recovers), len(sources), len(references))


recovers_S = list()
sources_S = list()
references_S = list()
for i in range(len(information)):
    epoch = information[i][0]
    recovers = information[i][1]
    sources = information[i][2]
    references = information[i][3]

    recovers_S.append(recovers)
    sources_S.append(sources)
    references_S.append(references)

print(len(information))

bleu, rougel, bert_score, dist1, selfBleu, div4, avg_len = metrics_calculate_S(recovers_S, sources_S, references_S, S)
bleu_list.append(bleu)
rougel_list.append(rougel)
bert_score_list.append(bert_score)
dist1_list.append(dist1)
selfBleu_list.append(selfBleu)
div4_list.append(div4)
avg_len_list.append(avg_len)

del recovers_S, sources_S, references_S, information, recovers, sources, references

e_t = time.time() - s_t
print(f'[Evaluate] Using {e_t/60:.2f} mins.')

10 ['/content/drive/MyDrive/RD資料夾/DiffusionOutput/BalaGinjo_checkpoint/Rsparse/test_inferences_ema_34000_seed6.csv', '/content/drive/MyDrive/RD資料夾/DiffusionOutput/BalaGinjo_checkpoint/Rsparse/test_inferences_ema_34000_seed9.csv', '/content/drive/MyDrive/RD資料夾/DiffusionOutput/BalaGinjo_checkpoint/Rsparse/test_inferences_ema_34000_seed525.csv', '/content/drive/MyDrive/RD資料夾/DiffusionOutput/BalaGinjo_checkpoint/Rsparse/test_inferences_ema_34000_seed64.csv', '/content/drive/MyDrive/RD資料夾/DiffusionOutput/BalaGinjo_checkpoint/Rsparse/test_inferences_ema_34000_seed25.csv', '/content/drive/MyDrive/RD資料夾/DiffusionOutput/BalaGinjo_checkpoint/Rsparse/test_inferences_ema_34000_seed52.csv', '/content/drive/MyDrive/RD資料夾/DiffusionOutput/BalaGinjo_checkpoint/Rsparse/test_inferences_ema_34000_seed95.csv', '/content/drive/MyDrive/RD資料夾/DiffusionOutput/BalaGinjo_checkpoint/Rsparse/test_inferences_ema_34000_seed65.csv', '/content/drive/MyDrive/RD資料夾/DiffusionOutput/BalaGinjo_checkpoint/Rsparse/test_infer

  0%|          | 0/4 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/2 [00:00<?, ?it/s]

done in 2.19 seconds, 58.50 sentences/sec
[Evaluate] Using 0.31 mins.


In [ ]:
for i in range(len(epochs)):
  print(f'epoch: {epochs[i]}')
  print(f'bleu: {bleu_list[i]:.4f},\nrougel: {rougel_list[i]:.4f},\nbert score: {bert_score_list[i]:.4f},\ndist1: {dist1_list[i]:.4f},\nselfBleu: {selfBleu_list[i]:.4f},\ndiv4: {div4_list[i]:.4f},\navg_len: {avg_len_list[i]:.2f}')
  print()

epoch: 34000
bleu: 0.0127,
rougel: 0.1071,
bert score: 0.4850,
dist1: 0.8727,
selfBleu: 0.0170,
div4: 0.9996,
avg_len: 17.76



In [ ]:
# for Gu check MBR ver
# MBR
# ver = 'Rdiff'
ver = 'Rsparse'
# ver = 'Rscale'
# ver = 'beta_ins_onlyZero'
# ver = 'onlyD'
checkpoint_path = f'/content/drive/MyDrive/RD資料夾/DiffusionOutput/BalaGinjo_checkpoint/{ver}'

bleu_list = list()
rougel_list = list()
bert_score_list = list()
dist1_list = list()
selfBleu_list = list()
div4_list = list()
avg_len_list = list()

# for Gu check
# 1. epochs, 2. use_ema
epoch = 52000
use_ema = False
# seed_list = [6, 9, 525, 64, 25, 52, 95, 65, 50, 7]
# seed_list = [640, 601, 949, 383, 821, 367, 248, 872, 963, 577]
# seed_list = [50, 65, 821, 6, 640, 9, 525, 601, 25, 52, 7, 963, 64, 383, 248]
seed_list = [6, 640]
S = len(seed_list)

s_t = time.time()

filepath = list()
for seed in seed_list:
  if use_ema:
    # filepath = [f'{checkpoint_path}/test_inferences_ema_52000_seed{seed}.csv']
    filepath.append(f'{checkpoint_path}/test_inferences_ema_52000_seed{seed}.csv')
  else:
    # filepath = [f'{checkpoint_path}/test_inferences_52000_seed{seed}.csv']
    filepath.append(f'{checkpoint_path}/test_inferences_52000_seed{seed}.csv')
  # print(len(filepath), filepath)

information = {}
count = 0
for file in filepath:
  with open(file, 'r', newline='') as csvfile:
      rows = csv.reader(csvfile)
      recovers = []
      sources = []
      references = []
      for row in rows:
          if f'Epoch' in row[0]:
              epoch = row[0].replace('\n', '')
              continue
          temp = row[0].replace('\n', '').replace('  ', '').replace('\'', '').replace('\"', '').split(',source:')
          recovers.append(temp[0].split('recover:')[1])
          sources.append(temp[1].split(',reference:')[0])
          references.append(temp[1].split(',reference:')[1])
      information[count] = [epoch, recovers, sources, references]
      count += 1
      # print(len(recovers), len(sources), len(references))


recovers_S = list()
sources_S = list()
references_S = list()
for i in range(len(information)):
    epoch = information[i][0]
    recovers = information[i][1]
    sources = information[i][2]
    references = information[i][3]

    recovers_S.append(recovers)
    sources_S.append(sources)
    references_S.append(references)

# print(len(information))

bleu = bleu_calculate_S(recovers_S, sources_S, references_S, S)
# print(f'seed: {seed} -> bleu: {bleu:.4f}')
print(f'BLEU: {bleu:.4f}')

del recovers_S, sources_S, references_S, information, recovers, sources, references

e_t = time.time() - s_t
print(f'[Evaluate] Using {e_t/60:.2f} mins.')

BLEU: 0.2864
[Evaluate] Using 0.00 mins.


In [ ]:
# MBR
# ver = 'original/QQP_sqrt_NCmean_2000'
ver = 'Rspase'
checkpoint_path = f'/content/drive/MyDrive/RD資料夾/DiffusionOutput/BalaGinjo_checkpoint/{ver}'

epochs = [i*1000 for i in range(50, 50+1)]

seed_list = [104, 66, 999, 168, 87, 555, 97, 888, 777, 108]
S = len(seed_list)

bleu_list = list()
rougel_list = list()
bert_score_list = list()
dist1_list = list()
selfBleu_list = list()
div4_list = list()
avg_len_list = list()

filepath = list()
for seed in seed_list:
  filepath.append(f'{checkpoint_path}/seed{seed}_step0.json')
print(filepath)

information = dict()
count = 0
for file in filepath:
  recovers = list()
  sources = list()
  references = list()
  with open(file, 'r', newline='') as file:
      for line in file:
          data=json.loads(line)
          recovers.append(data['recover'])
          references.append(data['reference'])
          sources.append(data['source'])
          # if len(recovers) >= 128:
          #     break
  information[count] = [recovers, sources, references]
  count += 1
  print(len(recovers), len(sources), len(references))
print(len(information))

recovers_S = list()
sources_S = list()
references_S = list()
for i in range(len(information)):
    recovers = information[i][0]
    sources = information[i][1]
    references = information[i][2]

    recovers_S.append(recovers)
    sources_S.append(sources)
    references_S.append(references)

bleu, rougel, bert_score, dist1, selfBleu, div4, avg_len = metrics_calculate_S(recovers_S, sources_S, references_S, S)

bleu_list.append(bleu)
rougel_list.append(rougel)
bert_score_list.append(bert_score)
dist1_list.append(dist1)
selfBleu_list.append(selfBleu)
div4_list.append(div4)
avg_len_list.append(avg_len)

['/content/drive/MyDrive/RD資料夾/DiffusionOutput/BalaGinjo_checkpoint/original/CC_sqrt_100/seed104_step0.json', '/content/drive/MyDrive/RD資料夾/DiffusionOutput/BalaGinjo_checkpoint/original/CC_sqrt_100/seed66_step0.json', '/content/drive/MyDrive/RD資料夾/DiffusionOutput/BalaGinjo_checkpoint/original/CC_sqrt_100/seed999_step0.json', '/content/drive/MyDrive/RD資料夾/DiffusionOutput/BalaGinjo_checkpoint/original/CC_sqrt_100/seed168_step0.json', '/content/drive/MyDrive/RD資料夾/DiffusionOutput/BalaGinjo_checkpoint/original/CC_sqrt_100/seed87_step0.json', '/content/drive/MyDrive/RD資料夾/DiffusionOutput/BalaGinjo_checkpoint/original/CC_sqrt_100/seed555_step0.json', '/content/drive/MyDrive/RD資料夾/DiffusionOutput/BalaGinjo_checkpoint/original/CC_sqrt_100/seed97_step0.json', '/content/drive/MyDrive/RD資料夾/DiffusionOutput/BalaGinjo_checkpoint/original/CC_sqrt_100/seed888_step0.json', '/content/drive/MyDrive/RD資料夾/DiffusionOutput/BalaGinjo_checkpoint/original/CC_sqrt_100/seed777_step0.json', '/content/drive/MyDri

  0%|          | 0/4 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/2 [00:00<?, ?it/s]

done in 1.93 seconds, 66.26 sentences/sec


In [ ]:
for i in range(len(epochs)):
  print(f'epoch: {epochs[i]}')
  print(f'bleu: {bleu_list[i]:.4f},\nrougel: {rougel_list[i]:.4f},\nbert score: {bert_score_list[i]:.4f},\ndist1: {dist1_list[i]:.4f},\nselfBleu: {selfBleu_list[i]:.4f},\ndiv4: {div4_list[i]:.4f},\navg_len: {avg_len_list[i]:.2f}')
  print()

epoch: 140000
bleu: 0.0166,
rougel: 0.1317,
bert score: 0.5217,
dist1: 0.9572,
selfBleu: 0.0141,
div4: 1.0000,
avg_len: 8.72



In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
ver = 'original/QQP_sqrt_NCmean_2000'
checkpoint_path = f'/content/drive/MyDrive/RD資料夾/DiffusionOutput/BalaGinjo_checkpoint/{ver}'

epochs = [i*1000 for i in range(21, 50+1)]

bleu_list = list()
rougel_list = list()
bert_score_list = list()
dist1_list = list()
selfBleu_list = list()
div4_list = list()
avg_len_list = list()

for epoch in epochs:
  files = sorted(glob.glob(f'{checkpoint_path}/inferences_epoch_ema_{epoch}.json'))
  # print(len(files), files)

  S = 1

  information = dict()
  count = 0
  for filepath in files:
      recovers = list()
      sources = list()
      references = list()
      with open(filepath, 'r', newline='') as file:
          for line in file:
              data=json.loads(line)
              recovers.append(data['recover'])
              references.append(data['reference'])
              sources.append(data['source'])
              # if len(recovers) >= 128:
              #     break
      information[count] = [recovers, sources, references]
      count += 1
      print(filepath)
      print(len(recovers), len(sources), len(references))
  print(len(information))

  recovers_S = list()
  sources_S = list()
  references_S = list()
  for i in range(len(information)):
      recovers = information[i][0]
      sources = information[i][1]
      references = information[i][2]

      recovers_S.append(recovers)
      sources_S.append(sources)
      references_S.append(references)

  bleu, rougel, bert_score, dist1, selfBleu, div4, avg_len = metrics_calculate_S(recovers_S, sources_S, references_S, S)

  bleu_list.append(bleu)
  rougel_list.append(rougel)
  bert_score_list.append(bert_score)
  dist1_list.append(dist1)
  selfBleu_list.append(selfBleu)
  div4_list.append(div4)
  avg_len_list.append(avg_len)

  del recovers_S, sources_S, references_S, information, recovers, sources, references

/content/drive/MyDrive/RD資料夾/DiffusionOutput/BalaGinjo_checkpoint/original/QQP_sqrt_NCmean_2000/inferences_epoch_ema_21000.json
128 128 128
1


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/792 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

...

done in 1.75 seconds, 73.26 sentences/sec
/content/drive/MyDrive/RD資料夾/DiffusionOutput/BalaGinjo_checkpoint/original/QQP_sqrt_NCmean_2000/inferences_epoch_ema_50000.json
128 128 128
1
calculating scores...
computing bert embedding.


  0%|          | 0/4 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/2 [00:00<?, ?it/s]

done in 1.72 seconds, 74.40 sentences/sec


In [ ]:
for i in range(len(epochs)):
  print(f'epoch: {epochs[i]}')
  print(f'bleu: {bleu_list[i]:.4f},\nrougel: {rougel_list[i]:.4f},\nbert score: {bert_score_list[i]:.4f},\ndist1: {dist1_list[i]:.4f},\nselfBleu: {selfBleu_list[i]:.4f},\ndiv4: {div4_list[i]:.4f},\navg_len: {avg_len_list[i]:.2f}')
  print()

epoch: 21000
bleu: 0.0736,
rougel: 0.3653,
bert score: 0.6078,
dist1: 0.8845,
selfBleu: 0.0000,
div4: 1.0000,
avg_len: 13.75

epoch: 22000
bleu: 0.0999,
rougel: 0.4066,
bert score: 0.6490,
dist1: 0.8981,
selfBleu: 0.0000,
div4: 1.0000,
avg_len: 13.02

epoch: 23000
bleu: 0.1095,
rougel: 0.4280,
bert score: 0.6800,
dist1: 0.9120,
selfBleu: 0.0000,
div4: 1.0000,
avg_len: 12.28

epoch: 24000
bleu: 0.1261,
rougel: 0.4538,
bert score: 0.7006,
dist1: 0.9212,
selfBleu: 0.0000,
div4: 1.0000,
avg_len: 11.77

epoch: 25000
bleu: 0.1385,
rougel: 0.4643,
bert score: 0.7182,
dist1: 0.9297,
selfBleu: 0.0000,
div4: 1.0000,
avg_len: 11.37

epoch: 26000
bleu: 0.1546,
rougel: 0.4832,
bert score: 0.7381,
dist1: 0.9371,
selfBleu: 0.0000,
div4: 1.0000,
avg_len: 11.09

epoch: 27000
bleu: 0.1660,
rougel: 0.4893,
bert score: 0.7489,
dist1: 0.9420,
selfBleu: 0.0000,
div4: 1.0000,
avg_len: 11.05

epoch: 28000
bleu: 0.1691,
rougel: 0.4984,
bert score: 0.7577,
dist1: 0.9466,
selfBleu: 0.0000,
div4: 1.0000,
avg_len:

In [ ]:
ver = 'Rnorm_oneStep'
# ver = 'beta_ins_onlyZero'
checkpoint_path = f'/content/drive/MyDrive/RD資料夾/DiffusionOutput/BalaGinjo_checkpoint/{ver}'
epoch = 3200

use_ema = True

if use_ema:
    filepath = f'{checkpoint_path}/test_inferences_ema_{epoch}.csv'
else:
    filepath = f'{checkpoint_path}/test_inferences_{epoch}.csv'
print(filepath)

S = 1

/content/drive/MyDrive/RD資料夾/DiffusionOutput/BalaGinjo_checkpoint/Rnorm_oneStep/test_inferences_ema_3200.csv


In [ ]:
information = {}
with open(filepath, 'r', newline='') as csvfile:
    rows = csv.reader(csvfile)
    recovers = []
    sources = []
    references = []
    for row in rows:
        if f'Epoch' in row[0]:
            epoch = row[0].replace('\n', '')
            continue
        temp = row[0].replace('\n', '').replace('  ', '').replace('\'', '').replace('\"', '').split(',source:')
        recovers.append(temp[0].split('recover:')[1])
        sources.append(temp[1].split(',reference:')[0])
        references.append(temp[1].split(',reference:')[1])
    information[0] = [epoch, recovers, sources, references]
print(len(recovers), len(sources), len(references))

128 128 128


In [ ]:
recovers_S = list()
sources_S = list()
references_S = list()
for i in range(len(information)):
    epoch = information[i][0]
    recovers = information[i][1]
    sources = information[i][2]
    references = information[i][3]

    recovers_S.append(recovers)
    sources_S.append(sources)
    references_S.append(references)
# print(np.array(recovers_S).shape)

s_t = time.time()

#     bleu, rougel, bert_score, dist1, avg_len = metrics_calculate(recovers, sources, references)
#     msg = f'bleu:{bleu:.4f},rougel:{rougel:.4f},bert score:{bert_score:.4f},dist1:{dist1:.4f},avg_len:{avg_len:.4f}'
#     record_metric_score(msg, f'{ver}_score')

bleu, rougel, bert_score, dist1, selfBleu, div4, avg_len = metrics_calculate_S(recovers_S, sources_S, references_S, S)
#     print(f'bleu:{bleu:.4f},\nrougel:{rougel:.4f},\nbert score:{bert_score:.4f},\ndist1:{dist1:.4f},\nselfBleu:{selfBleu:.4f},\ndiv4:{div4:.4f},\navg_len:{avg_len:.4f}')
#     msg = f'bleu:{bleu:.4f},rougel:{rougel:.4f},bert score:{bert_score:.4f},dist1:{dist1:.4f},selfBleu:{selfBleu:.4f},div4:{div4:.4f},avg_len:{avg_len:.4f}'
#     record_metric_score(msg, f'{ver}_score')

e_t = time.time() - s_t
print(f'[Evaluate] Using {e_t/60:.2f} mins.')
del recovers_S, sources_S, references_S, information, recovers, sources, references

calculating scores...
computing bert embedding.


  0%|          | 0/2 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/2 [00:00<?, ?it/s]

done in 2.99 seconds, 42.74 sentences/sec
[Evaluate] Using 0.52 mins.


In [ ]:
print(filepath.split('BalaGinjo_checkpoint')[1])
print(epoch)
print(f'bleu:{bleu:.4f},\nrougel:{rougel:.4f},\nbert score:{bert_score:.4f},\ndist1:{dist1:.4f},\nselfBleu:{selfBleu:.4f},\ndiv4:{div4:.4f},\navg_len:{avg_len:.2f}')
# print(f'{bleu:.4f}\t{rougel:.4f}\t{bert_score:.4f}\t{dist1:.4f}\t{selfBleu:.4f}\t{div4:.4f}\t{avg_len:.2f}')

/Rnorm_oneStep/test_inferences_ema_3200.csv
Epoch 3200 / -
bleu:0.0000,
rougel:0.0000,
bert score:0.0000,
dist1:0.0000,
selfBleu:0.0000,
div4:0.0000,
avg_len:1.00


In [ ]:
ver = 'original/QG100'
checkpoint_path = f'/content/drive/MyDrive/RD資料夾/DiffusionOutput/BalaGinjo_checkpoint/{ver}'
epoch = 40000

# filepath = f'{checkpoint_path}/inferences/seed123_step0_epoch{epoch}.json'
# filepath = f'{checkpoint_path}/inferences/seed103_step0.json'
# print(filepath)

files = sorted(glob.glob(f'{checkpoint_path}/inferences_epoch_{epoch}.json'))
print(len(files), files)

S = 1

1 ['/content/drive/MyDrive/RD資料夾/DiffusionOutput/BalaGinjo_checkpoint/original/QG100/inferences_epoch_40000.json']


In [ ]:
information = dict()
count = 0
for filepath in files:
    recovers = list()
    sources = list()
    references = list()
    with open(filepath, 'r', newline='') as file:
        for line in file:
            data=json.loads(line)
            recovers.append(data['recover'])
            references.append(data['reference'])
            sources.append(data['source'])
            # if len(recovers) >= 128:
            #     break
    information[count] = [recovers, sources, references]
    count += 1
    print(filepath)
    print(len(recovers), len(sources), len(references))
print(len(information))

/content/drive/MyDrive/RD資料夾/DiffusionOutput/BalaGinjo_checkpoint/original/QG100/inferences_epoch_40000.json
128 128 128
1


In [ ]:
recovers_S = list()
sources_S = list()
references_S = list()
for i in range(len(information)):
    recovers = information[i][0]
    sources = information[i][1]
    references = information[i][2]

    recovers_S.append(recovers)
    sources_S.append(sources)
    references_S.append(references)

s_t = time.time()

#     bleu, rougel, bert_score, dist1, avg_len = metrics_calculate(recovers, sources, references)
#     msg = f'bleu:{bleu:.4f},rougel:{rougel:.4f},bert score:{bert_score:.4f},dist1:{dist1:.4f},avg_len:{avg_len:.4f}'
#     record_metric_score(msg, f'{ver}_score')

bleu, rougel, bert_score, dist1, selfBleu, div4, avg_len = metrics_calculate_S(recovers_S, sources_S, references_S, S)
#     print(f'bleu:{bleu:.4f},\nrougel:{rougel:.4f},\nbert score:{bert_score:.4f},\ndist1:{dist1:.4f},\nselfBleu:{selfBleu:.4f},\ndiv4:{div4:.4f},\navg_len:{avg_len:.4f}')
#     msg = f'bleu:{bleu:.4f},rougel:{rougel:.4f},bert score:{bert_score:.4f},dist1:{dist1:.4f},selfBleu:{selfBleu:.4f},div4:{div4:.4f},avg_len:{avg_len:.4f}'
#     record_metric_score(msg, f'{ver}_score')

e_t = time.time() - s_t
print(f'[Evaluate] Using {e_t/60:.2f} mins.')
del recovers_S, sources_S, references_S, information, recovers, sources, references

calculating scores...
computing bert embedding.


  0%|          | 0/4 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/2 [00:00<?, ?it/s]

done in 2.25 seconds, 56.79 sentences/sec
[Evaluate] Using 0.42 mins.


In [ ]:
print(files[0].split('BalaGinjo_checkpoint/')[1])
# print(f'== epoch {epoch} ==')
print(f'bleu:{bleu:.4f},\nrougel:{rougel:.4f},\nbert score:{bert_score:.4f},\ndist1:{dist1:.4f},\nselfBleu:{selfBleu:.4f},\ndiv4:{div4:.4f},\navg_len:{avg_len:.2f}')

original/QG100/inferences_epoch_40000.json
bleu:0.0819,
rougel:0.4430,
bert score:0.7136,
dist1:0.9567,
selfBleu:0.0000,
div4:0.7109,
avg_len:9.80


In [ ]:
MBR
bleu:0.1912,
rougel:0.4019,
bert score:0.6431,
dist1:0.9031,
selfBleu:0.0000,
div4:0.9568,
avg_len:10.48


seed101_step0.json
bleu:0.1283,
rougel:0.3223,
bert score:0.5715,
dist1:0.9117,
selfBleu:0.0000,
div4:0.9709,
avg_len:11.05

seed102_step0.json
bleu:0.1301,
rougel:0.3208,
bert score:0.5684,
dist1:0.9088,
selfBleu:0.0000,
div4:0.9699,
avg_len:11.07

seed103_step0.json
bleu:0.1268,
rougel:0.3227,
bert score:0.5697,
dist1:0.9055,
selfBleu:0.0000,
div4:0.9678,
avg_len:11.07